# Cosmos DB Synapse Sales Processing Spark Notebook

## This Spark/PySpark Notebook demonstrates how to:

- **Read the Synapse Link Analytic Datastore with Spark/PySpark in Azure Synapse**
- **Source Cosmos DB is the Mongo API**
- **Aggregating the sales data by customer_id**
- Displaying the "shape" of the dataframes, and observed schema
- Filter the sales data (by doctype, timestamp)
- Writing the aggregated "materialized view" of sales-by-customer to the Cosmos DB views container


In [55]:
# Define common PySpark functions used in this Notebook

def print_df_shape(df, msg):
    print("shape of df {} - row count: {}, column count: {}".format(
        msg, str(df.count()), str(len(df.columns)) ))


StatementMeta(sparkpool3m, 9, 25, Finished, Available)

In [56]:
# Read the Synapse Link Analytic Datastore into a Spark Dataframe

from pyspark.sql.functions import col
import pyspark.sql.functions as F 

# initialize variables; these can be computed values
# such as for a daily or monthly report.
min_doc_timestamp = 1640995200  # 2022-01-01T00:00:00.000Z
max_doc_timestamp = 1999999999  # distant future

# Note: "cosmos.olap" is Synapse Link, "cosmos.oltp" is Cosmos DB

sales_df = spark.read\
    .format("cosmos.olap")\
    .option("spark.synapse.linkedService", "gbbcjmongo_retail")\
    .option("spark.cosmos.container", "sales")\
    .load()\
    .filter(col("_ts") >= min_doc_timestamp)\
    .filter(col("_ts") <= max_doc_timestamp)

display(sales_df.limit(10))


StatementMeta(sparkpool3m, 9, 26, Finished, Available)

SynapseWidget(Synapse.DataFrame, 97a9c13f-85e3-42bd-ac47-fbaff82e26e5)

In [57]:
# Display the shape and the schema of the input dataframe

print_df_shape(sales_df, "sales_df")
sales_df.printSchema()


StatementMeta(sparkpool3m, 9, 27, Finished, Available)

shape of df sales_df - row count: 108062, column count: 21
root
 |-- _rid: string (nullable = true)
 |-- _ts: long (nullable = true)
 |-- id: string (nullable = true)
 |-- _etag: string (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- objectId: string (nullable = true)
 |-- pk: struct (nullable = true)
 |    |-- string: string (nullable = true)
 |-- sale_id: struct (nullable = true)
 |    |-- int32: integer (nullable = true)
 |-- doctype: struct (nullable = true)
 |    |-- string: string (nullable = true)
 |-- date: struct (nullable = true)
 |    |-- string: string (nullable = true)
 |-- line_num: struct (nullable = true)
 |    |-- int32: integer (nullable = true)
 |-- customer_id: struct (nullable = true)
 |    |-- int32: integer (nullable = true)
 |-- store_id: struct (nullable = true)
 |    |-- int32: integer (nullable = true)
 |-- upc: struct (nullable = true)
 |    |-- string: string (nullable = true)
 |-- price: struct (nullable = true)
 |    |-- float64: double (nu

In [58]:
from pyspark.sql.functions import col
import pyspark.sql.functions as F 

# The input documents look like this; we'll select just a few
# of these attributes for processing.

# {
# 	"_id" : ObjectId("64458d9959d30378b040b2a5"),
# 	"pk" : "1",
# 	"id" : "051424eb-9de0-4ff4-810a-522cdd9ece07",
# 	"sale_id" : 1,
# 	"doctype" : "line_item",
# 	"date" : "2021-01-01",
# 	"line_num" : 1,
# 	"customer_id" : 6168,
# 	"store_id" : 60,
# 	"upc" : "0760981980837",
# 	"price" : 84.65,
# 	"qty" : 1,
# 	"cost" : 84.65,
# 	"epoch" : 1682279833.3812423
# }

sales_df_unpacked = sales_df.select(
    col('sale_id.*'),
    col('customer_id.*'),
    col('doctype.*'),
    col('item_count.*'),
    col('total_cost.*'))

# Rename the columns of the unpacked DataFrame to friendly names
new_column_names = ['sale_id', 'customer_id', 'doctype', 'item_count', 'total_cost']
df_sales = sales_df_unpacked.toDF(*new_column_names).filter(col("doctype") == "sale")

print_df_shape(df_sales, "df_sales")
display(df_sales.limit(10))
df_sales.printSchema()


StatementMeta(sparkpool3m, 9, 28, Finished, Available)

shape of df df_sales - row count: 30840, column count: 5


SynapseWidget(Synapse.DataFrame, 399f576c-46dc-4f78-999c-0c5f06952536)

root
 |-- sale_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- doctype: string (nullable = true)
 |-- item_count: integer (nullable = true)
 |-- total_cost: double (nullable = true)



In [59]:
# Aggregate Sales by Customer 

# But first, consider an alternative implementation using JUST CosmosDB, and not Spark:
# 1. Read customers container to get the unique set of customer ids (cross partition)
# 2. Loop through the customer id list:
#    - Read all of the sale documents for each customer in the timeframe (cross partition)
#    - sum the sales item_count and item_count for each customer (memory intensive)

import pyspark.sql.functions as F 

df_customer_aggregated = df_sales.groupBy("customer_id") \
    .agg(
        F.first('customer_id').alias('id'), \
        F.first('customer_id').alias('pk'), \
        F.count("customer_id").alias('order_count'), \
        F.sum("total_cost").alias("total_dollar_amount"), \
        F.sum("item_count").alias("total_item_count")) \
        .sort("customer_id", ascending=True)

print_df_shape(df_customer_aggregated, "df_customer_aggregated")
display(df_customer_aggregated.limit(10))
df_customer_aggregated.printSchema()


StatementMeta(sparkpool3m, 9, 29, Finished, Available)

shape of df df_customer_aggregated - row count: 9532, column count: 6


SynapseWidget(Synapse.DataFrame, 050f089a-0c94-444f-8812-5a699fce6ab3)

root
 |-- customer_id: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- pk: integer (nullable = true)
 |-- order_count: long (nullable = false)
 |-- total_dollar_amount: double (nullable = true)
 |-- total_item_count: long (nullable = true)



In [60]:
# Display Installed Packages

import pkg_resources
pkg_list = list()

for d in sorted(pkg_resources.working_set):
    pkg_list.append(str(d))
for p in sorted(pkg_list):
    pass
    #print(p)


StatementMeta(sparkpool3m, 9, 30, Finished, Available)

In [61]:
df_customer_aggregated.createOrReplaceTempView('agg')
print('df_customer_aggregated saved to tmp view: agg')


StatementMeta(sparkpool3m, 9, 31, Finished, Available)

df_customer_aggregated saved to tmp view: agg


In [62]:
%%sql

select * from agg limit 3


StatementMeta(sparkpool3m, 9, 32, Finished, Available)

<Spark SQL result set with 3 rows and 6 fields>

In [63]:
%%pyspark
df_for_cosmos = spark.sql("SELECT String(id), String(pk), order_count, total_dollar_amount, total_item_count FROM agg")

print_df_shape(df_for_cosmos, "df_for_cosmos")
display(df_for_cosmos.limit(10))
df_for_cosmos.printSchema()



StatementMeta(sparkpool3m, 9, 33, Finished, Available)

shape of df df_for_cosmos - row count: 9532, column count: 5


SynapseWidget(Synapse.DataFrame, 339ea633-a69b-43b3-ac62-91f30bbb139a)

root
 |-- id: string (nullable = true)
 |-- pk: string (nullable = true)
 |-- order_count: long (nullable = false)
 |-- total_dollar_amount: double (nullable = true)
 |-- total_item_count: long (nullable = true)



In [64]:

# See https://spark.apache.org/docs/3.2.4/api/python/reference/api/pyspark.sql.DataFrameWriter.mode.html

df_for_cosmos.write.format("cosmos.oltp")\
    .option("spark.synapse.linkedService", "gbbcjcdbnosql_retail_db")\
    .option("spark.cosmos.container", "views")\
    .mode('append')\
    .save()




StatementMeta(sparkpool3m, 9, 34, Finished, Available)

In [65]:
%%spark 

// Obtain configuration values from spark.conf
// See https://learn.microsoft.com/en-us/azure/synapse-analytics/spark/apache-spark-azure-create-spark-configuration

val connStr  = spark.conf.get("spark.azurepg.jdbc.connstring")
val driver   = spark.conf.get("spark.azurepg.jdbc.driver")
val server   = spark.conf.get("spark.azurepg.jdbc.server")
val table    = "public.customer_sales"
val user     = spark.conf.get("spark.azurepg.jdbc.user")
val password = spark.conf.get("spark.azurepg.jdbc.pass")

//println("connStr:  " + server)
println("driver:   " + driver)
println("server:   " + server)
println("database: " + database)
println("table:    " + table)
println("user:     " + user)
println("password: " + password)

val df_customer_aggregated = spark.read.table("agg")

println("df_customer_aggregated row count: " + df_customer_aggregated.count())

// Using JDBC, write the Dataframe to Azure PostgreSQL
df_customer_aggregated.write
  .format("jdbc")
  .option("url", connStr)
  .option("driver", driver)
  .option("dbtable", table)
  .option("user", "chjoakim")
  .option("password", password)
  .mode("overwrite")
  .save()

println("done\n\n\n\n")


StatementMeta(sparkpool3m, 9, 35, Finished, Available)

driver:   org.postgresql.Driver
server:   gbbcjflexpg.postgres.database.azure.com
database: postgres
table:    public.customer_sales
user:     chjoakim@gbbcjflexpg.postgres.database.azure.com
password: PX58VSv9DWZc
df_customer_aggregated row count: 9532
done




connStr: String = jdbc:postgresql://gbbcjflexpg.postgres.database.azure.com:5432/postgres?user=chjoakim&password=PX58VSv9DWZc&sslmode=require
driver: String = org.postgresql.Driver
server: String = gbbcjflexpg.postgres.database.azure.com
table: String = public.customer_sales
user: String = chjoakim@gbbcjflexpg.postgres.database.azure.com
password: String = PX58VSv9DWZc
df_customer_aggregated: org.apache.spark.sql.DataFrame = [customer_id: int, id: int ... 4 more fields]
